In [ ]:
!pwd

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import os
import pandas as pd
import seaborn as sns
import numpy
from tqdm import tqdm
np.random.seed(1)
from minisom import MiniSom
from sklearn.cluster import KMeans
import itertools
import gc
import networkx as nx
from collections import Counter
from matplotlib.gridspec import GridSpec
import matplotlib.cm as cm
from time import time
from sklearn import metrics


# import warnings
# warnings.filterwarnings("ignore")

In [ ]:

from minisom import MiniSom
import bisect
from itertools import combinations

class ConsensusCluster:
    """
      Implementation of Consensus clustering, following the paper
      https://link.springer.com/content/pdf/10.1023%2FA%3A1023949509487.pdf
      Args:
        * cluster -> clustering class
                    needs fit_predict method called with parameter n_clusters
        * L -> smallest number of clusters to try
        * K -> biggest number of clusters to try
        * H -> number of resamplings for each cluster number
        * resample_proportion -> percentage to sample
        * Mk -> consensus matrices for each k (shape =(K,data.shape[0],data.shape[0]))
                (NOTE: every consensus matrix is retained, like specified in the paper)
        * Ak -> area under CDF for each number of clusters 
                (see paper: section 3.3.1. Consensus distribution.)
        * deltaK -> changes in ares under CDF
                (see paper: section 3.3.1. Consensus distribution.)
        * self.bestK -> number of clusters that was found to be best
      """

    def __init__(self, cluster, L, K, H, resample_proportion=0.5):
        assert 0 <= resample_proportion <= 1, "proportion has to be between 0 and 1"
        self.cluster_ = cluster
        self.resample_proportion_ = resample_proportion
        self.L_ = L
        self.K_ = K
        self.H_ = H
        self.Mk = None
        self.Ak = None
        self.deltaK = None
        self.bestK = None

    def _internal_resample(self, data, proportion):
        """
        Args:
          * data -> (examples,attributes) format
          * proportion -> percentage to sample
        """
        resampled_indices = np.random.choice(
            range(data.shape[0]), size=int(data.shape[0]*proportion), replace=False)
        return resampled_indices, data[resampled_indices, :]

    def fit(self, data, verbose=False):
        """
        Fits a consensus matrix for each number of clusters
        Args:
          * data -> (examples,attributes) format
          * verbose -> should print or not
        """
        Mk = np.zeros((self.K_-self.L_, data.shape[0], data.shape[0]))
        Is = np.zeros((data.shape[0],)*2)
        for k in range(self.L_, self.K_):  # for each number of clusters
            i_ = k-self.L_
            if verbose:
                print("At k = %d, aka. iteration = %d" % (k, i_))
            for h in range(self.H_):  # resample H times
                if verbose:
                    print("\tAt resampling h = %d, (k = %d)" % (h, k))
                resampled_indices, resample_data = self._internal_resample(
                    data, self.resample_proportion_)
                Mh = self.cluster_(n_clusters=k).fit_predict(resample_data)
                # find indexes of elements from same clusters with bisection
                # on sorted array => this is more efficient than brute force search
                id_clusts = np.argsort(Mh)
                sorted_ = Mh[id_clusts]
                for i in range(k):  # for each cluster
                    ia = bisect.bisect_left(sorted_, i)
                    ib = bisect.bisect_right(sorted_, i)
                    is_ = id_clusts[ia:ib]
                    ids_ = np.array(list(combinations(is_, 2))).T
                    # sometimes only one element is in a cluster (no combinations)
                    if ids_.size != 0:
                        Mk[i_, ids_[0], ids_[1]] += 1
                # increment counts
                ids_2 = np.array(list(combinations(resampled_indices, 2))).T
                Is[ids_2[0], ids_2[1]] += 1
            Mk[i_] /= Is+1e-8  # consensus matrix
            # Mk[i_] is upper triangular (with zeros on diagonal), we now make it symmetric
            Mk[i_] += Mk[i_].T
            Mk[i_, range(data.shape[0]), range(
                data.shape[0])] = 1  # always with self
            Is.fill(0)  # reset counter
        self.Mk = Mk
        # fits areas under the CDFs
        self.Ak = np.zeros(self.K_-self.L_)
        for i, m in enumerate(Mk):
            hist, bins = np.histogram(m.ravel(), density=True)
            self.Ak[i] = np.sum(h*(b-a)
                             for b, a, h in zip(bins[1:], bins[:-1], np.cumsum(hist)))
        # fits differences between areas under CDFs
        self.deltaK = np.array([(Ab-Aa)/Aa if i > 2 else Aa
                                for Ab, Aa, i in zip(self.Ak[1:], self.Ak[:-1], range(self.L_, self.K_-1))])
        self.bestK = np.argmax(self.deltaK) + \
            self.L_ if self.deltaK.size > 0 else self.L_

    def predict(self):
        """
        Predicts on the consensus matrix, for best found cluster number
        """
        assert self.Mk is not None, "First run fit"
        return self.cluster_(n_clusters=self.bestK).fit_predict(
            1-self.Mk[self.bestK-self.L_])

    def predict_data(self, data):
        """
        Predicts on the data, for best found cluster number
        Args:
          * data -> (examples,attributes) format 
        """
        assert self.Mk is not None, "First run fit"
        return self.cluster_(n_clusters=self.bestK).fit_predict(
            data)
    
    def set_bestK(k):
        self.bestK = k

In [ ]:
dem_info = pd.read_excel('/work/zg78/brain-flow-data/All_demographic_data_abridged dm.xlsx')
dem_info

In [ ]:
dem_info['binary_druguse'] = dem_info['Group'].map({'dual':1, 'coc':1, 'mj':0, 'non':0})
dem_info

In [ ]:
print(np.unique(dem_info['binary_druguse'].values, return_counts=True))

In [ ]:
npy_path = '/work/zg78/brain-flow-data/CSF_viable_npy/'

# cell_events_per_subset = 10000

x_train = np.load(npy_path+os.listdir(npy_path)[0])
# subsample_idx = np.random.choice(x_train.shape[0], cell_events_per_subset, replace=False)
x_train_sub = x_train

for f in os.listdir(npy_path)[1:]:
    x_train_ = np.load(npy_path+f)

#     subsample_idx_ = np.random.choice(x_train_.shape[0], cell_events_per_subset, replace=False)
    x_train_sub_ = x_train_       
    x_train_sub = np.concatenate((x_train_sub, x_train_sub_), axis=0)

mean_ = np.mean(x_train_sub, axis=0)
std_ = np.std(x_train_sub, axis=0)
x_train_sub = (x_train_sub - mean_) / std_

print(x_train_sub.shape)

np.random.shuffle(x_train_sub)

size = 100
som = MiniSom(size, size, x_train_sub.shape[1],
              neighborhood_function='bubble', sigma=2, learning_rate=.8, topology='hexagonal',
              random_seed=42)
# som = MiniSom(size, size, x_train_sub.shape[1],
#               neighborhood_function='triangle', sigma=2, learning_rate=.8, topology='hexagonal',
#               random_seed=42)

som.pca_weights_init(x_train_sub)
som.train_random(x_train_sub, 10000, verbose=True)
weights = som.get_weights()
flatten_weights = weights.reshape(size*size, x_train_sub.shape[1])

cluster_ = ConsensusCluster(KMeans, 6, 25, 100, resample_proportion=0.9)
cluster_.fit(flatten_weights, True) # fitting SOM weights into clustering algorithm
print(flatten_weights.shape, weights.shape)
best_k = cluster_.bestK
print('K:', best_k)

In [ ]:
# import pickle as pkl
# os.mkdir('/work/zg78/som_kmeans_save/')
# with open('/work/zg78/som_kmeans_save/som.pkl', 'wb') as somf:
#     pkl.dump(som, somf)
    
# with open('/work/zg78/som_kmeans_save/km.pkl', 'wb') as kmf:
#     pkl.dump(cluster_, kmf)

In [ ]:
import pickle as pkl
with open('/work/zg78/som_kmeans_save/som.pkl', 'rb') as somf:
    som = pkl.load(somf)
    
with open('/work/zg78/som_kmeans_save/km.pkl', 'rb') as kmf:
    cluster_ = pkl.load(kmf)

In [ ]:
npy_path = '/work/zg78/brain-flow-data/CSF_viable_npy/'

x_train = np.load(npy_path+os.listdir(npy_path)[0])
x_train_sub = x_train

for f in os.listdir(npy_path)[1:]:
    x_train_ = np.load(npy_path+f)

    x_train_sub_ = x_train_
    x_train_sub = np.concatenate((x_train_sub, x_train_sub_), axis=0)

mean_ = np.mean(x_train_sub, axis=0)
std_ = np.std(x_train_sub, axis=0)
x_train_sub = (x_train_sub - mean_) / std_

size = 100
weights = som.get_weights()
flatten_weights = weights.reshape(size*size, x_train_sub.shape[1])

best_k = cluster_.bestK
print('K:', best_k)
flatten_class = cluster_.predict_data(flatten_weights)
map_class = flatten_class.reshape(size, size)
print(np.unique(map_class, return_counts=True))

In [ ]:
from multiprocessing import Pool


    
def get_row_rst(som_, map_class, xx):
    winner = som_.winner(xx)
    c = map_class[winner] # from the location info get cluster info
    return c

def get_classes(som_, x_test, k):

    x_test = (x_test - np.mean(x_test, axis=0)) / np.std(x_test, axis=0)
    
    pool_args = []
    for i in range(len(x_test)):
        xx = x_test[i, :]
        pool_args.append([som, map_class, xx])
    
    pool = Pool()
    label_list = pool.starmap(get_row_rst, pool_args)
    pool.terminate()

    label_counts = [0]*k
    label, counts = np.unique(np.asarray(label_list), return_counts=True)
    for i in range(len(label)):
        label_counts[label[i]] = counts[i]

    return np.asarray(label_counts) / x_test.shape[0]

npy_path = '/work/zg78/brain-flow-data/CSF_viable_npy/'

kf = KFold(n_splits=5, shuffle=True, random_state=1)
for fidx, splits in enumerate(kf.split(os.listdir(npy_path))):
    
    xs_train = []
    ys_train = []
    scores_train = []
    xs_val = []
    ys_val = []
    scores_val = []

    covars_train = []
    covars_val = []
    
    train_idx, val_idx = splits
    train_f = [os.listdir(npy_path)[i] for i in train_idx]
    val_f = [os.listdir(npy_path)[i] for i in val_idx]
    
    for f in tqdm(train_f):
        subject = int(f.split(' ')[0])
        if subject == 1001: subject = 2001
        x = np.load(npy_path + f)
        x = (x - mean_) / std_
        rst = get_classes(som, x, best_k)
        y = dem_info.loc[dem_info['PID'] == subject]['binary_druguse'].values
        xs_train.append(rst)
        ys_train.append(y[0])
        
    
    for f in tqdm(val_f):
        subject = int(f.split(' ')[0])
        if subject == 1001: subject = 2001
        x = np.load(npy_path + f)
        x = (x - mean_) / std_
        rst = get_classes(som, x, best_k)
        y = dem_info.loc[dem_info['PID'] == subject]['binary_druguse'].values
        xs_val.append(rst)
        ys_val.append(y[0])
    
    xs_train = np.asarray(xs_train)
    ys_train = np.asarray(ys_train)
    xs_val = np.asarray(xs_val)
    ys_val = np.asarray(ys_val)
    
    print(xs_train.shape, ys_train.shape, xs_val.shape, ys_val.shape)

    np.save('/work/zg78/som_folds/CSF100_X_train_{}.npy'.format(fidx), xs_train)
    np.save('/work/zg78/som_folds/CSF100_y_train_{}.npy'.format(fidx), ys_train)
    np.save('/work/zg78/som_folds/CSF100_X_val_{}.npy'.format(fidx), xs_val)
    np.save('/work/zg78/som_folds/CSF100_y_val_{}.npy'.format(fidx), ys_val)

In [ ]:
def get_row_rst(som_, map_class, xx):
    winner = som_.winner(xx)
    c = map_class[winner] # from the location info get cluster info
    return c

x = np.load(npy_path+os.listdir(npy_path)[0])
for f in tqdm(os.listdir(npy_path)[1:]):
    x_ = np.load(npy_path+f)
    x = np.concatenate((x, x_))
x = (x - np.mean(x, axis=0)) / np.std(x, axis=0)
print(x.shape)
cluster_labels = []
for row in tqdm(x):
    cluster = get_row_rst(som, map_class, row)
    cluster_labels.append(cluster)

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
np.random.seed(1)
subsample_idx = np.random.choice(x.shape[0], 5000, replace=False)
cluster_labels = np.asarray(cluster_labels)
print(np.unique(cluster_labels))

tsne = TSNE(n_components=2)
tsne_x = tsne.fit_transform(x[subsample_idx])
# print(np.unique(cluster_labels))
plt.figure(figsize=(16,10))

plt.scatter(
    x=tsne_x[:, 0], y=tsne_x[:, 1],
    c=cluster_labels[subsample_idx],
    alpha=0.8
)


In [ ]:
cog_info = pd.read_excel('/work/zg78/brain-flow-data/BRAIN NP+WCST_Merged 2020-0528_abridged dm.xlsx')
cog_info

In [ ]:
dem_info_nadropped = dem_info.dropna()
cog_info_nadropped = cog_info.dropna()

In [ ]:
def get_dem_info(categories):
    dem_infos = []
    for cat in categories:
        info = dem_info.loc[dem_info['PID'] == subject][cat].values[0]
        dem_infos.append(info)
    
    return dem_infos

dem_info_categories = ['cigdyc_r', 'agey_np', 'edu_np', 'Years.HIV', 'Years.Treat', 'MDE_lifetime', 'CoMorbid']

kf = KFold(n_splits=5, shuffle=True, random_state=1)
for fidx, splits in enumerate(kf.split(os.listdir(npy_path))):

    covars_train = []
    covars_val = []
    
    cog_train = []
    cog_val = []

    train_idx, val_idx = splits
    train_f = [os.listdir(npy_path)[i] for i in train_idx]
    val_f = [os.listdir(npy_path)[i] for i in val_idx]
    
    for f in tqdm(train_f):
        subject = int(f.split(' ')[0])
        if subject == 1001: subject = 2001
        dem_infos = get_dem_info(dem_info_categories)
        cog_status = cog_info.loc[cog_info['subject'] == subject]['Global_T_impair'].values[0]
        covars_train.append(dem_infos)
        cog_train.append(cog_status)
    
    for f in tqdm(val_f):
        subject = int(f.split(' ')[0])
        if subject == 1001: subject = 2001
        dem_infos = get_dem_info(dem_info_categories)
        cog_status = cog_info.loc[cog_info['subject'] == subject]['Global_T_impair'].values[0]
        covars_val.append(dem_infos)
        cog_val.append(cog_status)
    
    covars_train = np.asarray(covars_train)
    covars_val = np.asarray(covars_val)
    
#     covar_mean_ = np.mean(covars_train, axis=0)
#     covar_std_ = np.std(covars_train, axis=0)
#     covars_train = (covars_train - covar_mean_) / covar_std_
#     covars_val = (covars_val - covar_mean_) / covar_std_
    
    cog_train = np.asarray(cog_train)
    cog_val = np.asarray(cog_val)
    
    np.save('/work/zg78/som_folds/covar_train_{}.npy'.format(fidx), covars_train)
    np.save('/work/zg78/som_folds/cog_train_{}.npy'.format(fidx), cog_train)
    np.save('/work/zg78/som_folds/covar_val_{}.npy'.format(fidx), covars_val)
    np.save('/work/zg78/som_folds/cog_val_{}.npy'.format(fidx), cog_val)